# Import Packages

In [1]:
# Method #2 HAR-X Model
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm

# Method #3 Regularisation Model
from sklearn.linear_model import ElasticNet

from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
from set_params import func_train_test_split, count_train_test
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd
import warnings

def calculate_iqr(values):
    # Calculate Q1
    Q1 = np.percentile(values, 25)
    # Calculate Q3
    Q3 = np.percentile(values, 75)
    # Calculate IQR
    IQR = Q3 - Q1
    return IQR

def detect_outliers_iqr(values):
    # Calculate the IQR of the values
    IQR = calculate_iqr(values)
    # Calculate Q1 and Q3
    Q1 = np.percentile(values, 25)
    Q3 = np.percentile(values, 75)
    # Define the lower and upper bound for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # Return a boolean array: True if the value is an outlier, False otherwise
    return lower_bound, upper_bound

def vis_line_plot_results(y_pred, y_test, model = 'HAR', name = 'BARCLAYS', r = 1, dataset = 'm1'):

    plt.figure(figsize=(10,4))
    true, = plt.plot(y_test)
    preds, = plt.plot(y_pred)
    plt.title(f'{model}-{dataset}-{name}', fontsize=15)
    plt.legend(['True Volatility', 'Predicted Volatility'], fontsize=9)
    plt.xticks(rotation=45)
    plt.savefig(f'../outputs/{model}-{dataset}/{str(r+1).zfill(3)}-{model}-{name}.png')
    plt.close()

# display(HTML("<style>.container { width:80% !important; }</style>"))
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)

# Data Processing

## Import Data and Split Train - Test

In [2]:
base_FTSE_df = pd.read_csv('../data/1.3-FTSE_Monthly_ESG_Volatility_Final.csv')
base_FTSE_df = base_FTSE_df.rename(columns={'Date_x':'date_key'})

In [3]:
train_df, valid_df, test_df = func_train_test_split(validation = False, threshold = 24)

In [4]:
count_rows_df = count_train_test(train_df, test_df)

In [5]:
coverage_df = pd.read_csv('../data/coverage_dataframe.csv')
coverage_df.PermID = coverage_df.PermID.astype(int)
coverage_df = coverage_df[['PermID', 'Name']]
coverage_df = coverage_df.rename(columns={'PermID':'Asset'})

In [6]:
train_df = pd.merge(train_df, coverage_df, how = 'left', on = 'Asset')
train_df.index = train_df.month_key

In [7]:
train_df.head(3)

,date_key,month_key,Asset,Open,High,Low,Close,Return,buzz,ESG,ESGCombined,ESGControversies,EnvironmentalPillar,GovernancePillar,SocialPillar,CSRStrategy,Community,Emissions,EnvironmentalInnovation,HumanRights,Management,ProductResponsibility,ResourceUse,Shareholders,Workforce,V^CC,V^RS,V^YZ,vol_series_daily,vol_series_weekly,vol_series_monthly,Name
month_key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-12-01,2021-01-29,2020-12-01,5039731355,33.500000,33.500000,33.000000,33.300004,-0.017699,444.5,54.0,31.0,8.0,88.0,53.0,40.0,0.0,56.0,69.058824,88.0,0.0,53.0,5.0,44.934247,50.0,33.778082,0.015634,0.015792,0.017582,NaN,NaN,NaN,Pershing Square Holdings Ltd
2021-01-01,2021-02-26,2021-01-01,5039731355,35.399994,35.800004,34.350007,34.600007,-0.036211,444.5,54.0,31.0,8.0,88.0,53.0,40.0,0.0,56.0,69.058824,88.0,0.0,53.0,5.0,44.934247,50.0,33.778082,0.020611,0.014774,0.018086,0.017582,NaN,NaN,Pershing Square Holdings Ltd
2021-02-01,2021-03-31,2021-02-01,5039731355,35.949997,35.949997,35.149994,35.500000,0.004243,487.5,27.0,17.0,7.0,81.0,24.0,5.0,0.0,6.0,66.946269,81.0,0.0,15.0,4.0,44.934247,51.0,33.778082,0.022377,0.016992,0.019119,0.018086,NaN,NaN,Pershing Square Holdings Ltd


- Buzz score (1)
- ESG Overall score (1)
- ESG Combined score (1)
- ESG Controversy score (1)
- Pillar scores (3)
- Category scores (10)

In total there are 17 scores provided by MP ESG Core

In [8]:
cols = [
    'buzz','ESG','ESGCombined','ESGControversies','EnvironmentalPillar','GovernancePillar','SocialPillar'
                ,'CSRStrategy','Community','Emissions','EnvironmentalInnovation','HumanRights','Management','ProductResponsibility'
                ,'ResourceUse','Shareholders','Workforce', 'vol_series_daily','vol_series_weekly','vol_series_monthly', 'V^YZ']

In [9]:
train_df[cols].shape

(13214, 21)

# Modeling

## Create Functions

In [8]:
def compile_train_test(train_df, test_df, sample = True, algo = 'HAR', dataset = 'm1', viz = False, cap = True):
    '''
    '''

    if dataset == 'm1':
        cols = ['V^YZ', 'vol_series_daily', 'vol_series_weekly', 'vol_series_monthly']

    elif dataset == 'm3':
        cols = [ 'buzz','ESG','ESGCombined','ESGControversies','EnvironmentalPillar','GovernancePillar','SocialPillar'
                ,'CSRStrategy','Community','Emissions','EnvironmentalInnovation','HumanRights','Management','ProductResponsibility'
                ,'ResourceUse','Shareholders','Workforce', 'vol_series_daily','vol_series_weekly','vol_series_monthly', 'V^YZ']

    mresults = pd.DataFrame()

    if sample:
        assets = [4295894970, 8589934212]
    else:
        assets = train_df.Asset.unique().tolist()

    for r, asset in enumerate(assets): 

        name = train_df[train_df['Asset'] == asset].iloc[0,-1]

        df_train = train_df[train_df.Asset == asset][cols].dropna()
        df_test = test_df[test_df.Asset == asset][cols].dropna()
        indices = test_df[test_df.Asset == asset].index
        
        test_size = df_test.shape[0]
        
        X_train = df_train.drop(['V^YZ'], axis=1)
        X_test = df_test.drop(['V^YZ'], axis=1)
        
        y_train = df_train['V^YZ']
        y_test = df_test['V^YZ']

        # Fit the model
        model = ElasticNet(l1_ratio = 0.5)
        model_fit = model.fit(X_train, y_train)

        # display(X_test, X_train)
        y_pred = model_fit.predict(X_test)
        y_pred = pd.Series(y_pred, index=indices)
        
        if cap:
            y_pred = y_pred.clip(lower = 0)
        
        mse_million = mean_squared_error(y_test,y_pred)*10**3
        mresult = pd.DataFrame({
            'Asset': asset,
            'Name': name,
            'Model': algo,
            'Test Size': test_size,
            'MSE^3':mse_million
                    }
            , index=[r]
        )
        mresults = pd.concat([mresults, mresult])

        if viz: 
            vis_line_plot_results(y_pred, y_test, model = algo, dataset=dataset, name=name, r = r)

    return mresults

## Datasets M1 (Basic Lag Value)

In [38]:
mresults_m1 = compile_train_test(train_df, test_df, sample=False, algo='EN', dataset='m1', viz=True, cap = True)

In [39]:
mresults_m1.sort_values('MSE^3', ascending=False)

,Asset,Name,Model,Test Size,MSE^3
112,4295895499,HBOS Plc,EN,10,5.327473
70,5000683618,Currys PLC,EN,12,3.382616
91,5000047647,Friends Life FPG Ltd,EN,12,1.538333
59,4295896428,Autonomy Corp Ltd,EN,10,1.481896
35,4295894092,Travis Perkins PLC,EN,12,1.303628
...,...,...,...,...,...
9,4295894930,Spirax-Sarco Engineering PLC,EN,14,0.012831
69,4295893850,G4S Ltd,EN,31,0.012128
28,4298449570,Direct Line Insurance Group PLC,EN,17,0.012022
64,4295895691,Alliance Trust PLC,EN,10,0.011038


In [40]:
np.mean(mresults_m1['MSE^3'])

0.20204156202434195

In [42]:
MODEL = 'EN'
THRESHOLD = str(24)
CAP = '0CAP'
mresults_m1.to_excel(f'../results/1-{MODEL}-{THRESHOLD}MONTH-{CAP}.xlsx', index=None)

## Datasets m3 (Overall Data)

In [47]:
mresults_m3 = compile_train_test(train_df, test_df, sample=False, algo='EN', dataset='m3', viz=True, cap = True)

In [44]:
mresults_m3.sort_values('MSE^3', ascending=False)

,Asset,Name,Model,Test Size,MSE^3
112,4295895499,HBOS Plc,EN,10,5.327473
70,5000683618,Currys PLC,EN,12,3.382616
91,5000047647,Friends Life FPG Ltd,EN,12,1.484658
59,4295896428,Autonomy Corp Ltd,EN,10,1.481896
35,4295894092,Travis Perkins PLC,EN,12,1.303628
...,...,...,...,...,...
9,4295894930,Spirax-Sarco Engineering PLC,EN,14,0.012831
28,4298449570,Direct Line Insurance Group PLC,EN,17,0.012022
64,4295895691,Alliance Trust PLC,EN,10,0.011038
23,4295874865,DCC PLC,EN,25,0.008525


In [45]:
np.mean(mresults_m3['MSE^3'])

0.2235014147615777

In [42]:
MODEL = 'EN'
THRESHOLD = str(24)
CAP = '0CAP'
mresults_m1.to_excel(f'../results/1-{MODEL}-{THRESHOLD}MONTH-{CAP}.xlsx', index=None)

---